# In-Memory Cache

In [10]:
import os

os.environ['OPENAI_API_KEY'] = '###################'

Get your ChatOpenAI Instance ready

In [11]:
import langchain
from langchain.chat_models import ChatOpenAI

llm  = ChatOpenAI()

Setup In-Memory Cache

In [12]:
from langchain.cache import InMemoryCache

langchain.llm_cache = InMemoryCache()

Ask a question and measure how long it takes for the LLM to respond

In [15]:
%%time
llm.invoke("What is OpenAI?")

CPU times: user 735 µs, sys: 93 µs, total: 828 µs
Wall time: 811 µs


AIMessage(content='OpenAI is an artificial intelligence research laboratory and company that aims to ensure that artificial general intelligence (AGI) benefits all of humanity. It was founded in December 2015 as a non-profit organization, with the goal of advancing AI research and sharing the benefits with the wider society. OpenAI has since transitioned to a for-profit company, but still maintains a commitment to its mission of ensuring that AGI is used for the benefit of all. The organization conducts research in various areas of AI, publishes most of its findings, and develops AI technologies and models such as GPT-3 (Generative Pre-trained Transformer 3).')

Ask the same question again to see the quicker response

In [ ]:
%%time
llm.invoke("What is OpenAI?")

# SQLite Cache

In [6]:
!rm -f .cache.db

In [7]:
from langchain.cache import SQLiteCache

langchain.llm_cache = SQLiteCache(database_path = '.cache.db')

Ask a question and measure how long it takes for the LLM to respond

In [9]:
%%time
llm.invoke("What is OpenAI?")

CPU times: user 27.2 ms, sys: 12.7 ms, total: 39.9 ms
Wall time: 38.9 ms


AIMessage(content='OpenAI is an artificial intelligence research laboratory and company. It was founded in December 2015 with the goal of developing and promoting friendly AI that benefits all of humanity. OpenAI conducts research in various areas of artificial intelligence, including machine learning, natural language processing, computer vision, and robotics. The organization is known for its contributions to deep learning models such as GPT-3 and its commitment to ensuring the responsible and ethical development of AI technologies. OpenAI also aims to provide public goods and share research findings to foster collaboration and advance the field of AI.')

Ask the same question again to see the quicker response

In [ ]:
%%time
llm.invoke("What is OpenAI?")

# Semantic Cache

For this test, it's necessary to setup a local redis server listening on port 6379. You can use the following docker image to setup your redis instance by following the included instructions: https://hub.docker.com/r/redis/redis-stack-server/

In [43]:
from langchain_openai import OpenAI

llm = OpenAI(model_name="gpt-3.5-turbo-instruct", n=2, best_of=2)

Initialize the Redis semantic cache with default score threshold 0.2

In [54]:
from langchain_openai import OpenAIEmbeddings
from langchain.cache import RedisSemanticCache

langchain.llm_cache = RedisSemanticCache(redis_url="redis://localhost:6379", embedding=OpenAIEmbeddings(), score_threshold=0.2)

In [56]:
%%time
llm.invoke("What is OpenAI?")

CPU times: user 9.1 ms, sys: 2.72 ms, total: 11.8 ms
Wall time: 221 ms


'\n\nOpenAI is an artificial intelligence research organization founded in 2015 by entrepreneurs and researchers including Elon Musk, Sam Altman, and Ilya Sutskever. Its goal is to develop and promote friendly AI in a way that is safe and beneficial for humanity. OpenAI conducts research in various areas of AI, including machine learning, robotics, and natural language processing, and also creates and releases open-source AI tools and platforms for others to use. Its ultimate objective is to ensure that artificial intelligence is developed and used for the benefit of all people, rather than for the benefit of a small group or individual.'

Ask the same question again to see the quicker response

In [ ]:
%%time
llm.invoke("What is OpenAI?")

Adjusting the threshold value

In [46]:
from langchain.globals import set_llm_cache

set_llm_cache(RedisSemanticCache(
    redis_url="redis://localhost:6379",
    embedding=OpenAIEmbeddings(),
    score_threshold=0.5
))

# GPTCache

In [47]:
from gptcache import Cache
from gptcache.adapter.api import init_similar_cache
from langchain.cache import GPTCache
import hashlib

def get_hashed_name(name):
    return hashlib.sha256(name.encode()).hexdigest()

def init_gptcache(cache_obj: Cache, llm: str):
    hashed_llm = get_hashed_name(llm)
    init_similar_cache(cache_obj=cache_obj, data_dir=f"similar_cache_{hashed_llm}")

langchain.llm_cache = GPTCache(init_gptcache)

In [49]:
%%time
llm.invoke("What is OpenAI?")

CPU times: user 631 ms, sys: 4.51 ms, total: 635 ms
Wall time: 132 ms


'\n\nOpenAI is an artificial intelligence research organization founded in 2015 by a group of entrepreneurs, researchers, and philanthropists, including Elon Musk, Sam Altman, and Peter Thiel. Its mission is to develop and promote friendly AI for the benefit of humanity. OpenAI conducts research in various areas of artificial intelligence, including machine learning, robotics, and natural language processing. It also works on developing tools and technologies to make AI more accessible and beneficial to society. OpenAI is known for its cutting-edge research and has made significant contributions to the field of AI. Its work is published and shared publicly for the benefit of the wider AI community.'

Ask the same question again to see the quicker response

In [ ]:
%%time
llm.invoke("What is OpenAI?")